In [1]:
from utils import h36motion3d as datasets
from torch.utils.data import DataLoader
from Autoformer_main.models.Autoformer import Model as autoformer 
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.autograd
import torch
import numpy as np
from utils.loss_funcs import *
from utils.data_utils import define_actions
from utils.h36_3d_viz import *
import time

import torch.nn.functional as F
import torch.nn as nn
### To a better printing
from tabulate import tabulate

from utils import h36motion3d as datasets
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation

In [9]:
from  import *

ImportError: attempted relative import with no known parent package

In [2]:
# # Arguments to setup the datasets
datas = 'h36m' # dataset name
path = './data/h3.6m/h3.6m/dataset'
input_n=10 # number of frames to train on (default=10)
output_n=25 # number of frames to predict on
input_dim=3 # dimensions of the input coordinates(default=3)
skip_rate=1 # # skip rate of frames
joints_to_consider=22


#FLAGS FOR THE TRAINING
mode='train' #choose either train or test mode

batch_size_test=8
model_path= './checkpoints/' # path to the model checkpoint file

actions_to_consider_test='all' # actions to test on.
model_name = datas+'_3d_'+str(output_n)+'frames_ckpt' #the model name to save/load

#FLAGS FOR THE VISUALIZATION
actions_to_consider_viz='all' # actions to visualize
visualize_from='test'
n_viz=2

In [3]:
# Load Data
print('Loading Train Dataset...')
dataset = datasets.Datasets(path,input_n,output_n,skip_rate, split=0)
print('Loading Validation Dataset...')
vald_dataset = datasets.Datasets(path,input_n,output_n,skip_rate, split=1)

Loading Train Dataset...


c:\Users\paolo\Untitled Folder\AML\final_project\autoformer\utils\h36motion3d.py:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  subs = np.array([[1, 6, 7, 8, 9], [11], [5]]) # , 6, 7, 8, 9


Loading Validation Dataset...


In [4]:
batch_size=256

print('>>> Training dataset length: {:d}'.format(dataset.__len__()))
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)#

print('>>> Validation dataset length: {:d}'.format(vald_dataset.__len__()))
vald_loader = DataLoader(vald_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

>>> Training dataset length: 180077
>>> Validation dataset length: 28110


In [6]:
class Configs:
    def __init__(self, seq_len, pred_len, output_attention=False, moving_avg=3):
        self.seq_len = seq_len
        self.label_len = seq_len//2
        self.pred_len = pred_len
        self.output_attention = output_attention
        self.moving_avg = moving_avg
        self.enc_in=66
        self.dec_in=66
        self.d_model=66
        self.embed='fixed'
        self.freq='h'
        self.dropout=0.1
        self.e_layers=1
        self.d_layers=1
        self.c_out=66
        self.factor=1
        self.n_heads=1
        self.d_ff=16
        self.activation="relu"

In [7]:
configs = Configs(seq_len=10, pred_len=15)
autof = autoformer(configs)

In [8]:
# Arguments to setup the optimizer
lr=1e-04 # learning rate
use_scheduler=True # use MultiStepLR scheduler
milestones=[10,30]   # the epochs after which the learning rate is adjusted by gamma
gamma=0.1 #gamma correction to the learning rate, after reaching the milestone epochs
weight_decay=1e-05 # weight decay (L2 penalty)
optimizer=optim.Adam(autof.parameters(),lr=lr,weight_decay=weight_decay)
device = "cpu"

if use_scheduler:
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma)

clip_grad=None # select max norm to clip gradients
# Argument for training
n_epochs=41
log_step=200

In [9]:
def train(data_loader,vald_loader, path_to_save_model=None):
  train_loss = []
  val_loss = []
  val_loss_best = 1000

  dim_used = np.array([6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 21, 22, 23, 24, 25,
                    26, 27, 28, 29, 30, 31, 32, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
                    46, 47, 51, 52, 53, 54, 55, 56, 57, 58, 59, 63, 64, 65, 66, 67, 68,
                    75, 76, 77, 78, 79, 80, 81, 82, 83, 87, 88, 89, 90, 91, 92])

  for epoch in range(n_epochs-1):
      running_loss=0
      n=0
      autof.train()
      for cnt,batch in enumerate(data_loader):
          batch=batch.float().to(device)
          batch_dim=batch.shape[0]
          n+=batch_dim

          sequences_train=torch.cat((torch.zeros(*batch[:,:1,dim_used].size()).to(device),batch[:,1:10,dim_used]-batch[:,:9,dim_used]),1)
          sequences_gt=batch[:,10:25,dim_used]

          optimizer.zero_grad()
          sequences_predict, kl = autof(sequences_train)
          sequences_predict[:,1:15,:]=sequences_predict[:,1:15,:]+sequences_predict[:,:14,:]
          sequences_predict=sequences_predict+batch[:,9:10,dim_used]

          loss=mpjpe_error(sequences_predict,sequences_gt)

          if cnt % log_step == 0:
            print('[Epoch: %d, Iteration: %5d]  training mpjpe: %.3f, training kl: %.3f, training loss: %.3f' %(epoch + 1, cnt + 1, loss.item(), kl, loss.item()+kl))
          loss += kl

          loss.backward()
          if clip_grad is not None:
            torch.nn.utils.clip_grad_norm_(autof.parameters(),clip_grad)

          optimizer.step()
          running_loss += loss*batch_dim

      train_loss.append(running_loss.detach().cpu()/n)
      autof.eval()
      with torch.no_grad():
          running_loss=0
          n=0
          for cnt,batch in enumerate(vald_loader):
              batch=batch.float().to(device)
              batch_dim=batch.shape[0]
              n+=batch_dim

              sequences_train=torch.cat((torch.zeros(*batch[:,:1,dim_used].size()).to(device),batch[:,1:10,dim_used]-batch[:,:9,dim_used]),1)
              sequences_gt=batch[:,10:25,dim_used]

              sequences_predict, kl = autof(sequences_train)
              sequences_predict[:,1:15,:]=sequences_predict[:,1:15,:]+sequences_predict[:,:14,:]
              sequences_predict=sequences_predict+batch[:,9:10,dim_used]


              loss=mpjpe_error(sequences_predict,sequences_gt)

              if cnt % log_step == 0:
                print('[Epoch: %d, Iteration: %5d]  training mpjpe: %.3f, training kl: %.3f, training loss: %.3f' %(epoch + 1, cnt + 1, loss.item(), kl, loss.item()+kl))
              loss += kl

              running_loss+=loss*batch_dim
          val_loss.append(running_loss.detach().cpu()/n)
          if running_loss/n < val_loss_best:
            val_loss_best = running_loss/n

      if use_scheduler:
        scheduler.step()


In [11]:
train(data_loader,vald_loader)

[Epoch: 1, Iteration:     1]  training mpjpe: 79.593, training kl: 7.529, training loss: 87.121


KeyboardInterrupt: 